# Fast F1 - Save data

## Import libraries

In [1]:
import fastf1
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# ## TODO
# ## - Need to generalise better with the current date for ongoing season (schedule.iloc).
# ## - Maybe reorganise the columns when saving or better leave for later preprocessing? 
# ##   But might be unneccessary disk space if I won't use the other columns.
# ## - Remove preseason-testing event to not be saved.

# # Specify the season year you want to process
# year = 2024  # Replace with the desired season year

# # Retrieve the event schedule for the specified season
# schedule = fastf1.get_event_schedule(year)
# schedule = schedule.iloc[:22] 

# # Create an empty list to collect all laps data
# all_laps = []

# # Create an empty list to collect all results data
# all_results = []

# # Iterate over each event in the schedule
# for _, event in schedule.iterrows():
#     # Load the race session
#     session = fastf1.get_session(year, event['EventName'], 'R')
#     session.load()  # Load the session data
    
#     # Add the event name as a new column in the laps DataFrame
#     laps = session.laps
#     laps['EventName'] = event['EventName']
    
#     # Append the modified laps data to the list
#     all_laps.append(laps)
#     print(f"Laps data loaded for {event['EventName']}")

#     # Add the event name as a new column in the results DataFrame
#     results = session.results
#     results['EventName'] = event['EventName']

#     # Append the modified results data to the list
#     all_results.append(results)
#     print(f"Results data loaded for {event['EventName']}")

# # Concatenate all collected laps data into a single DataFrame
# if all_laps:
#     all_laps_df = pd.concat(all_laps, ignore_index=True)
    
#     # Save the DataFrame to a CSV file
#     output_file = f'data/gps_{year}_season_laps.csv'
#     all_laps_df.to_csv(output_file, index=False)
#     print(f"Laps data for {year} saved to {output_file}")
# else:
#     print("No laps data available for the selected season.")

# # Concatenate all collected results data into a single DataFrame
# if all_results:
#     all_results_df = pd.concat(all_results, ignore_index=True)
    
#     # Save the DataFrame to a CSV file
#     output_file = f'data/gps_{year}_season_results.csv'
#     all_results_df.to_csv(output_file, index=False)
#     print(f"Results data for {year} saved to {output_file}")
# else:
#     print("No results data available for the selected season.")

In [2]:
## TODO
## - Need to generalise better with the current date for ongoing season (schedule.iloc).
## - Maybe reorganise the columns when saving or better leave for later preprocessing? 
##   But might be unneccessary disk space if I won't use the other columns.
## - Remove preseason-testing event to not be saved.

# Specify the season year you want to process
year = 2024  # Replace with the desired season year

# Retrieve the event schedule for the specified season
schedule = fastf1.get_event_schedule(year)
schedule = schedule.iloc[:22] 

# Create an empty list to collect all results data
all_results = []

# Iterate over each event in the schedule
for _, event in schedule.iterrows():
    # Load the race session
    session = fastf1.get_session(year, event['EventName'], 'Q')
    session.load()  # Load the session data

    # Add the event name as a new column in the results DataFrame
    results = session.results
    results['EventName'] = event['EventName']

    # Append the modified results data to the list
    all_results.append(results)
    print(f"Results data loaded for {event['EventName']}")

# Concatenate all collected results data into a single DataFrame
if all_results:
    all_results_df = pd.concat(all_results, ignore_index=True)
    
    # Save the DataFrame to a CSV file
    output_file = f'data/gps_{year}_season_quali.csv'
    all_results_df.to_csv(output_file, index=False)
    print(f"Results data for {year} saved to {output_file}")
else:
    print("No results data available for the selected season.")




req         WARNING 	DEFAULT CACHE ENABLED! (632.1 MB) /home/michail/.cache/fastf1
events      WARNING 	Correcting user input 'Pre-Season Testing' to 'Singapore Grand Prix'
core           INFO 	Loading data for Singapore Grand Prix - Qualifying [v3.4.4]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO

Results data loaded for Pre-Season Testing


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

Results data loaded for Bahrain Grand Prix


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

...

Results data loaded for United States Grand Prix


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

Results data loaded for Mexico City Grand Prix


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

Results data loaded for São Paulo Grand Prix
Results data for 2024 saved to data/gps_2024_season_quali.csv
